## Use Case 2: Data from multiple sources in the animal lab
### This notebook illustrates the usage of the pyvital class to visualize, annotate and process time-series data from the medical field.
### In this case we analyze data from multiple sources from an animal lab experiment on cardiopulmonary resucitation. The main focus lies on aligning different time-series with clocks which are off.

In [1]:
from vitabel import Vitals, IntervalLabel
from pathlib import Path
import pandas as pd

## 1) Load Data

 We first specify the file paths of the files to read. In this use-case 3 different types of files are read. The data from the mechanical CPR device is stored within an xml-export from a proprietary format. Data about airflow, capnography and airway pressure are stored in csv-Files. Since the time in these Files is saved as unix-time, values for time_start, time_unit are set. 

In [4]:
Flow_file = Path("data/flow.csv.bz2")
capnography_file = "data/capno.csv.bz2"
airway_pressure_file = "data/p1.csv.bz2"
Lucas_file = Path("data/Lucas_file_Lucas.xml")
vital_file = "./data/vital_file.vit"

A new instance of the pyvital class is initialized and all data is loaded from the files afterward. 

In [5]:
case = Vitals()
case.add_defibrillator_recording(Lucas_file)
case.add_data_from_csv(
    Flow_file,
    time_start=pd.Timestamp(1970, 1, 1, 0, 0, 0),
    time_unit="ms",
    metadata={"source": "volucapno"},
    index_col="timestamp",
)
case.add_data_from_csv(
    capnography_file,
    time_start=pd.Timestamp(1970, 1, 1, 0, 0, 0),
    time_unit="ms",
    metadata={"source": "volucapno"},
    index_col="Timestamp",
)
case.add_data_from_csv(
    airway_pressure_file,
    time_start=pd.Timestamp(1970, 1, 1, 0, 0, 0),
    time_unit="ms",
    metadata={"source": "volucapno"},
    index_col=0,
    names=["airway_pressure", "temperature_1"],
)
case.add_vital_db_recording(vital_file, metadata={"source": "GE Healthcare monitor"})

 We get an overview over all channels by calling the info routine

In [ ]:
case.info()

## 2) Process Data

Several channels are called and renamed according to our terminology.

In [ ]:
capno_channel = case.get_channel("CO2 Concentration")
capno_channel.rename("capnography")
ibp_channel = case.get_channel("GE alt/IBP1")
case.get_channel("GE alt/PLETH").rename("ppg")

## 3) Interactively Plot Data


The remaining channels are called. 

In [ ]:
CC_channel = case.get_channel("cc")
flow_channel = case.get_channel("airflow")
pressure_channel = case.get_channel("airway_pressure")

 New Labels which should be annotated are initialized. For marking noisy segments of the invasive blood pressure, an IntervalLabel is used. 

In [ ]:
Blood_pressure_noise = IntervalLabel("Blood pressure noise")
Blood_pressure_noise.attach_to(ibp_channel)

Information about channels and labels and their attachment can be printed by the following routine

In [ ]:
case.print_data_summary()

The plotstyle of the channels is adapted by the set_channel_plotstyle function. All keyword-arguments of axes.plot in matplotlib can be stored in the plotstyle dictionary. 

In [ ]:
case.set_channel_plotstyle(capno_channel, color="goldenrod", lw=1, alpha=0.8)
case.set_channel_plotstyle(flow_channel, color="blue", lw=1, alpha=0.8)
case.set_channel_plotstyle(pressure_channel, color="green", lw=1, alpha=0.8)
case.set_channel_plotstyle(
    CC_channel, color="purple", marker="o", alpha=0.8, linestyle=""
)
case.set_channel_plotstyle(ibp_channel, color="red", lw=1, alpha=0.8)
case.set_label_plotstyle(Blood_pressure_noise, color="red", lw=3, alpha=0.4)

An interactive plot is initialized and the channels are shifted in the 'Adjust'-Tab. 

Afterward, we label noisy segments in the blood-pressure signal in the 'Annotate'-Framework.

In [ ]:
int_plot = case.plot_interactive(
    channels=[
        [flow_channel, CC_channel],
        [capno_channel],
        [pressure_channel],
        [ibp_channel],
    ],
    labels=[[], [], [], [Blood_pressure_noise]],
    channel_overviews=[[ibp_channel]],
    time_unit="s",
    subplots_kwargs={"figsize": (22, 9)},
)
int_plot

## 4) Store Data


In [ ]:
case.save_data("case_2.json")